In [1]:
from qiskit import *
from qiskit.tools.monitor import job_monitor
import numpy as np
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import AerSimulator
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
import qiskit.providers.aer.noise as noise
import time
# provider = IBMQ.load_account()

shots1toN = 500
# backend_sim = Aer.get_backend('qasm_simulator')
# backend_sim = Aer.get_backend('qasm_simulator',
#                             backend_options={'method': 'extended_stabilizer'})
extended_stabilizer_simulator = AerSimulator(method='extended_stabilizer')
# backend_sim = Aer.get_backend('extended_stabilizer')

ModuleNotFoundError: No module named 'qiskit.tools'

In [2]:
# a * b lattice
def generate_cir(a,b):
    n = a * b
    qr = range(n)
    cr = range(n)
    circuit = QuantumCircuit(n,n)
    circuit.h(range(n))
    
    for row_ite in range(a-1): #[1,a-2]
        for col_ite in range(b-1): #[1,b-2]
            cur = row_ite * b + col_ite
            circuit.cz(cur,cur + 1)
            circuit.cz(cur, cur + b)
    
    cur_col = b-1
    for ite in range(a-1):
        cur = ite * b + cur_col
        circuit.cz(cur, cur + b)
    cur_row = a-1
    for ite in range(b-1):
        cur = cur_row * b + ite
        circuit.cz(cur, cur + 1)
    return circuit

In [ ]:
circuit = generate_cir(2,2)
n = 2 * 2
circuit.draw(output = 'mpl')

In [ ]:
## count the parity of the key
def NumOnes(meas, key):
    n = len(key)
    count = 0
    for j in range(n):
        if meas[j+1] != 0 and key[n - j - 1]=='1':  ##inverse order for key
            count += 1
    return count  

## append circuit and measure
def sample_circuit(measure, a, b, prob_1, prob_2, prob_ro):
    n = a * b
    ite_max = len(measure)
    print('Number of measurements: %d'%ite_max)
    circs = []
    for cur in range(ite_max):
        circ = generate_cir(a,b)
        for cur_qb in range(n):
            if measure[cur][cur_qb + 1] == 1:
                circ.h(cur_qb)
            elif measure[cur][cur_qb + 1] == 2:
                circ.sdg(cur_qb)
                circ.h(cur_qb)


        # Depolarizing quantum errors
        error_1 = noise.depolarizing_error(prob_1, 1)
        error_2 = noise.depolarizing_error(prob_2, 2)

        # Add errors to noise model
        noise_model = noise.NoiseModel()
        noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3']) #all single-qubit gates
        noise_model.add_all_qubit_quantum_error(error_2, ['cz']) #depolarizing for two-qubit gates
        
        for qi in range(n):
            read_err = noise.errors.readout_error.ReadoutError([[1-prob_ro, prob_ro],[prob_ro, 1-prob_ro]])
            noise_model.add_readout_error(read_err, [qi])
        # Get basis gates from noise model
        basis_gates = noise_model.basis_gates
        
        circ.measure(range(n), range(n))
        circs.append(circ)
    

# #     circs = transpile(circs, backend=backend_sim, optimization_level=0)
#  #   job = execute(circs, backend_sim, basis_gates=basis_gates, noise_model=noise_model, shots=shots1toN)
#     job = execute(circs, backend_sim, shots=shots1toN)
# #     job = execute(circ, backend=backend_sim)

   
#     result = job.result()
    
#    result = qiskit.execute(circs, backend=Aer.get_backend('qasm_simulator', shots=shots1toN,
#                             backend_options={'method': 'extended_stabilizer'})).result()
    
    circs = transpile(circs, extended_stabilizer_simulator)
    job = extended_stabilizer_simulator.run(circs, shots = shots1toN)
    result = job.result()
    hit_basis = result.get_counts() 
    return hit_basis


## estimate
def estimation(measure, str_basis, val_basis, d_basis):
    len_key = len(str_basis)
    expect = 0 
    sum = 0
    count_basis = 0
    count_d = 0
    for j in range(len_key):
        ones = NumOnes(measure[count_basis], str_basis[j])
        count_d += 1
        parity = ones % 2
        outcome = 1 - 2 * parity
        outcome = outcome * measure[count_basis][0]* val_basis[j]
#         outcome = outcome * val_basis[j]
        sum = sum + val_basis[j]
#         print(measure[count_basis])
#         print(str_basis[j])
        expect = expect + outcome
#         print('---%d-th, count_basis = %d, and outcome = %d, expect = %d ' %(j,count_basis,outcome, expect))
        if count_d >= d_basis[count_basis]:
            count_d = 0
            count_basis += 1
    print('Summation of expect: %d'%expect)
    expect = float(expect)/sum
    print('With %d copies of state, the expectation value is %f' %(sum, expect))

In [ ]:
#read measurements from observables_16.txt
start = time.time()
a = 7
b = 7
n = a * b
# Error probabilities
prob_1 = 0.0016 # 1-qubit gate
prob_2 = 0.0062 # 2-qubit gate
prob_ro = 0.0038 # readoutError
fileName = 'observables_' + str(n) + '.txt'
print(fileName)
fileObj = open(fileName, "r") #opens the file in read mode
measure = [[int(x) for x in line.split()] for line in fileObj]
fileObj.close()
hit_basis = sample_circuit(measure, a,b, prob_1, prob_2, prob_ro)
arr = np.array(hit_basis)
str_basis = [key for p in arr for key in p.keys()]
val_basis = [p[key] for p in arr for key in p.keys()]
d_basis = [len(p) for p in arr]
estimation(measure, str_basis, val_basis,d_basis) #str_basis is a string array
# print(d_basis)
end = time.time()
print('Time cost: %.4f' %(end - start))

In [3]:
from qiskit.quantum_info import Clifford
from qiskit import *
# from qiskit.providers import job_monitor
import numpy as np
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

In [7]:
qc = generate_cir(7,7)
qc.measure(range(49), range(49))
extended_stabilizer_simulator = AerSimulator(method='extended_stabilizer')
job = extended_stabilizer_simulator.run(qc, shots = 1024)
result = job.result()
hit_basis = result.get_counts() 


In [8]:
print(hit_basis)

{'0001101100001111101001111101110101110011110000100': 1, '0100110010000110001111101111101101111001110010110': 1, '0011111110100110100001001000111011110101001111110': 1, '0100000011010011110100010011001001011111011110110': 1, '1101100100001000111000010011110110101110011100011': 1, '1101011010110011101110001010001011111111101010011': 1, '1001010110110001001111110100101000110101000100010': 1, '1100101100011110100110110111010110011101001000100': 1, '1100110111001001010001000010111001111010100010000': 1, '1001011111011010001010110111101001000111010001110': 1, '1110001100000101101010011101011000001010011101111': 1, '1001000010111101000011111000000000100001011011010': 1, '1111110001110011110011010011111001101110001010011': 1, '1010110010010000111000110100010000101100101111110': 1, '0100011110000000001001001101000111001000101110000': 1, '1110001010110101111011011100111011001111000001011': 1, '1111000100110010010101100010000010011011001111111': 1, '0011100111111011011001011101010100001000100110